# Create a Pipeline
We provide three ways of creating a pipeline.
* Functional API
* Imperative API
* Constructor API

In the following, we briefly describe all three APIs.

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# From pyWATTS the pipeline is imported
from pywatts.callbacks import LinePlotCallback

from pywatts_pipeline.core.steps.step import Step
from pywatts_pipeline.core.util.computation_mode import ComputationMode
from pywatts_pipeline.core.pipeline import Pipeline
# All modules required for the pipeline are imported
from pywatts.modules import CalendarExtraction, CalendarFeature, ClockShift, LinearInterpolater, SKLearnWrapper, Sampler
from pywatts.modules.preprocessing.select import Select
from pywatts.summaries import RMSE

C:\Users\bi4372\.conda\envs\pywatts_workshop\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\bi4372\.conda\envs\pywatts_workshop\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\bi4372\.conda\envs\pywatts_workshop\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Functional API

The functional API provides an easy way to create pipelines. However, it requires that the call dunder is implemented in the used transformers/modules, which is the case for pyWATTS transformers.
The API is inspired by the functional API of Keras. In general the notation is as follows:

```Transformer()(x=predeccessor, y=predecessor, ...)```

In the following, we show how a simple Pipeline can be created with the Functional API

In [ ]:
functional_api_pipeline = Pipeline(path="../results")

# Extract dummy calendar features, using holidays from Germany
# NOTE: CalendarExtraction can't return multiple features.

functional_preprocessing_pipeline = Pipeline()
calendar = CalendarExtraction(continent="Europe",
                              country="Germany",
                              features=[CalendarFeature.month, CalendarFeature.weekday,
                                        CalendarFeature.weekend]
                              )(x=functional_preprocessing_pipeline["load_power_statistics"])

# Deal with missing values through linear interpolation
imputer_power_statistics = LinearInterpolater(method="nearest", dim="time",
                                              name="imputer_power"
                                              )(x=functional_preprocessing_pipeline["load_power_statistics"])


scaler_target = SKLearnWrapper(module=StandardScaler(), name="scaled_target")
scaled_target = scaler_target(x=imputer_power_statistics)

# Scale the data using a standard SKLearn scaler
power_scaler = SKLearnWrapper(module=StandardScaler(), name="scaled_input")
scale_power_statistics = power_scaler(x=imputer_power_statistics)


added_prepro_pipe = functional_preprocessing_pipeline(load_power_statistics=functional_api_pipeline["load_power_statistics"])
# Create lagged time series to later be used as regressors
lag_features = Select(start=-2, stop=0, step=1)(x=added_prepro_pipe["scaled_input"])

target_multiple_output = Select(start=0, stop=24, step=1, name="sampled_data")(x=added_prepro_pipe["scaled_target"])

# Select features based on F-statistic
selected_features = SKLearnWrapper(
    module=SelectKBest(score_func=f_regression, k=2), name="kbest"
)(
    lag_features=lag_features,
    calendar=added_prepro_pipe["calendar"],
    target=added_prepro_pipe["scaled_input"],
)

# Create a linear regression that uses the lagged values to predict the current value
# NOTE: SKLearnWrapper has to collect all **kwargs itself and fit it against target.
#       It is also possible to implement a join/collect class
regressor_power_statistics = SKLearnWrapper(
    module=LinearRegression(fit_intercept=True)
)(
    features=selected_features,
    target=target_multiple_output,
    callbacks=[LinePlotCallback("linear_regression")],
)

# Rescale the predictions to be on the original time scale
inverse_power_scale = scaler_target(
    x=regressor_power_statistics, computation_mode=ComputationMode.Transform,
    method="inverse_transform", callbacks=[LinePlotCallback("rescale")]
)

# Calculate the root mean squared error (RMSE) between the linear regression and the true values
# save it as csv file
rmse = RMSE()(y_hat=inverse_power_scale, y=target_multiple_output)


In [ ]:
functional_api_pipeline.steps["RMSE_10"].input_steps["y_hat"]

In [ ]:
functional_api_pipeline.steps["scaled_target_9"].input_steps["x"].input_steps["features"].input_steps["lag_features"].input_steps["x"].input_steps["result"].buffer


## Imperative API

The imperative API is an alternative API for pyWATTS Pipelines. It can be used if the transformers do not implement a call dunder.
The general notation is as follows

```TODO```

In the following, we implement the same pipeline as above with a functional API.

In [ ]:
imperative_api_pipeline = Pipeline()

imperative_api_pipeline.add(
    CalendarExtraction(continent="Europe", country="Germany",
                       features=[CalendarFeature.month, CalendarFeature.weekday,
                                 CalendarFeature.weekend]),
    "calendar",
    {"x": "load_power_statistics"}
)

imperative_api_pipeline.add(
    LinearInterpolater(method="nearest", dim="time", name="imputer_power"),
    "imputer",
    {"x": "load_power_statistics"}
)
power_scaler = SKLearnWrapper(module=StandardScaler(), name="scaler_power")

imperative_api_pipeline.add(
    power_scaler,
    "scaler",
    {"x": "imputer"}
)

imperative_api_pipeline.add(
    Select(start=-2, stop=0, step=1),
    "lag_features",
    {"x": "scaler"}
)

imperative_api_pipeline.add(
    Select(start=0, stop=24, step=1),
    "target",
    {"x": "scaler"}
)

imperative_api_pipeline.add(
    SKLearnWrapper(module=SelectKBest(score_func=f_regression, k=2), name="kbest"),
    "selected_features",
    {"lag_features": "lag_features",
     "calendar": "calendar",
     "target": "scaler"}
)

imperative_api_pipeline.add(
    SKLearnWrapper(module=LinearRegression(fit_intercept=True)),
    "regression",
    {"selected_features": "selected_features",
     "target": "target"}
)

imperative_api_pipeline.add(
    power_scaler,
    "inverse_scaler",
    {"x": "regression"},
    method="inverse_transform",
    callbacks=[LinePlotCallback("rescale")],
    computation_mode=ComputationMode.Transform
)

imperative_api_pipeline.add(
    RMSE(),
    "rmse",
    {"y_hat": "inverse_scaler",
     "y": "target"},
)


In [ ]:
pipeline = Pipeline(
    steps=[
        (CalendarExtraction(continent="Europe", country="Germany",
                            features=[CalendarFeature.month, CalendarFeature.weekday,
                                      CalendarFeature.weekend]),
         "calendar",
         {"x": "load_power_statistics"}, {}),
        (LinearInterpolater(method="nearest", dim="time", name="imputer_power"),
         "imputer",
         {"x": "load_power_statistics"}, {}),
        (power_scaler,
         "scaler",
         {"x": "imputer"}, {}),
        (Select(start=-2, stop=0, step=1),
         "lag_features",
         {"x": "scaler"}, {}),
        (Select(start=0, stop=24, step=1),
         "target",
         {"x": "scaler"}, {}),
        (SKLearnWrapper(module=SelectKBest(score_func=f_regression, k=2), name="kbest"),
         "selected_features",
         {"lag_features": "lag_features",
          "calendar": "calendar",
          "target": "scaler"}, {}),
        (SKLearnWrapper(module=LinearRegression(fit_intercept=True)),
         "regression",
         {"selected_features": "selected_features",
          "target": "target"}, {}),
        (power_scaler,
         "inverse_scaler",
         {"x": "regression"},
         {"method": "inverse_transform",
          "callbacks": [LinePlotCallback("rescale")],
          "computation_mode": ComputationMode.Transform}),
        (RMSE(),
         "rmse",
         {"y_hat": "inverse_scaler",
          "y": "target"}, {})
    ]
)


In [ ]:

# Now, the pipeline is complete, so we can run it and explore the results
# Start the pipeline
data = pd.read_csv("../data/getting_started_data.csv",
                   index_col="time",
                   parse_dates=["time"],
                   infer_datetime_format=True,
                   sep=",")
train = data.iloc[:6000, :]
test = data.iloc[6000:, :]


In [ ]:
pipeline.train(data=train)
pipeline.test(data=test)

In [ ]:
from pywatts_pipeline.core.transformer.base import BaseEstimator

for s in pipeline.steps:
    if isinstance(s[0], Step) and isinstance(s[0].module, BaseEstimator):
        print(s[0].name, s[0].module.is_fitted, id(s[0].module))

In [ ]:
functional_api_pipeline.train(data=train)
functional_api_pipeline.test(data=test)

In [ ]:
imperative_api_pipeline.train(data=train)
imperative_api_pipeline.test(data=test)